In [1]:
import polars as pl

In [7]:
df = pl.read_csv('Dummy_EIR_Test_Data_3.csv')
df.head()

ASSESSMENT_DATE,ACCOUNT_NUMBER,CASH_FLOW_DATE,PRINCIPAL_AMOUNT,INTEREST_AMOUNT,REPAYMENT_AMOUNT,DISBURSED_AMOUNT,PARTIAL_AMOUNT,FEE_CHARGE,CONTRACTUAL_INTEREST_RATE,CURRENCY_ID,DAY_COUNT_BASIS
str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,i64
"""2023-02-27 18:…",624011221310001,"""2024-05-16 18:…",166.67,24.27,190.94,0.0,0.0,0.0,14.4,1.0,360
"""2023-02-27 18:…",624011221310001,"""2024-11-17 18:…",166.67,14.93,181.6,0.0,0.0,0.0,14.4,1.0,360
"""2023-02-27 18:…",624011221310001,"""2023-06-19 18:…",166.67,54.4,221.07,0.0,0.0,0.0,14.4,1.0,360
"""2023-02-27 18:…",624011221310001,"""2025-03-16 18:…",166.67,5.6,172.27,0.0,0.0,0.0,14.4,1.0,360
"""2023-02-27 18:…",624011221310001,"""2024-07-16 18:…",166.67,22.0,188.67,0.0,0.0,0.0,14.4,1.0,360


In [9]:
P = pl.Series([0] * len(df['REPAYMENT_AMOUNT']))

In [14]:
df['INTEREST_AMOUNT'][1]

14.93

In [12]:
type(P)

polars.series.series.Series

In [13]:
P[0] = -1 

0

In [2]:
def EIR(INT_RTE, Total_Repayment, Principle_disbu, Principle_prepay, Proces_fee, Day_count_Basis, No_of_days, retu_type):
    P = pl.Series([0] * len(Principle_disbu))
    A = pl.Series([0] * len(Principle_disbu))
    E = pl.Series([0] * len(Principle_disbu))

    P[0] = (-1 * Principle_disbu[0]) - Proces_fee[0]
    E[0] = (P[0] * INT_RTE) / Day_count_Basis[0] * (No_of_days[0])
    A[0] = E[0]

    for i in range(1, len(Principle_disbu)):
        if Total_Repayment[i] == 0:
            P[i] = P[i-1] - Proces_fee[i] - Principle_disbu[i] - Principle_prepay[i]
            E[i] = ((P[i-1]) * INT_RTE) / Day_count_Basis[i] * (No_of_days[i])
            A[i] = A[i-1] + E[i]
        else:
            E[i] = ((P[i-1]) * INT_RTE) / Day_count_Basis[i] * No_of_days[i]
            A[i] = 0
            P[i] = P[i-1] - (Proces_fee[i] + Principle_disbu[i] + Principle_prepay[i]) - (Total_Repayment[i] - A[i-1] - E[i])
    #print(P[-1])
    if retu_type == 1:
        return pl.DataFrame({"P": P, "E": E, "A": A})
    else:
        return P[-1]

In [3]:
def EIR(INT_RTE, Total_Repayment, Principle_disbu, Principle_prepay, Proces_fee, Day_count_Basis, No_of_days, retu_type):
    P = [0] * len(Principle_disbu)
    A = [0] * len(Principle_disbu)
    E = [0] * len(Principle_disbu)

    P[0] = (-1 * Principle_disbu[0]) - Proces_fee[0]
    E[0] = (P[0] * INT_RTE) / Day_count_Basis[0] * (No_of_days[0])
    A[0] = E[0]

    for i in range(1, len(Principle_disbu)):
        if Total_Repayment[i] == 0:
            P[i] = P[i-1] - Proces_fee[i] - Principle_disbu[i] - Principle_prepay[i]
            E[i] = ((P[i-1]) * INT_RTE) / Day_count_Basis[i] * (No_of_days[i])
            A[i] = A[i-1] + E[i]
        else:
            E[i] = ((P[i-1]) * INT_RTE) / Day_count_Basis[i] * No_of_days[i]
            A[i] = 0
            P[i] = P[i-1] - (Proces_fee[i] + Principle_disbu[i] + Principle_prepay[i]) - (Total_Repayment[i] - A[i-1] - E[i])
    #print(P[-1])
    if retu_type == 1:
        return pd.DataFrame({"P": P, "E": E, "A": A})
    else:
        return P[-1]


In [4]:
import polars as pl

def adjusted_intrest(INT_RTE, Total_Repayment, Principle_disbu, Principle_prepay, Proces_fee, Day_count_Basis, No_of_days, retu_type):
    int_r = INT_RTE # 0.18
    retr = EIR(INT_RTE,Total_Repayment,Principle_disbu,Principle_prepay,Proces_fee,Day_count_Basis
                              ,No_of_days,retu_type)
    X = 0
    M = 0
    for i in range (3,15):
        while retr < 0:
            M = int_r
            int_r  =  int_r + X/(10**i)
            retr = EIR(int_r,Total_Repayment,Principle_disbu,Principle_prepay,Proces_fee,Day_count_Basis
                              ,No_of_days,retu_type)
            X += 1

        int_r = M + (X-1)/(10**i)
        retr = EIR(int_r,Total_Repayment,Principle_disbu,Principle_prepay,Proces_fee,Day_count_Basis
                              ,No_of_days,retu_type)
        #print(f'Last retr value is {retr}')
    return (int_r)

In [5]:
    Y = P11.groupby('Month').agg(pl.col('rank').max())
    distinct_months = Y.to_series(1).unique()
    distinct_rank = Y.to_series(2)    
    
    dates = pl.Series([])
    period1_l = pl.Series([])
    period2_l = pl.Series([])
    period3_l = pl.Series([])
    period4_l = pl.Series([])
    period5_l = pl.Series([])
    period6_l = pl.Series([])
    expense_l = pl.Series([])
    expense_eir_l = pl.Series([])
    air_l = pl.Series([])
    air_eir_l = pl.Series([])
            
    for i in range(len(distinct_months)):
        filter_df = P11.filter(pl.col('Month') == distinct_months[i])
        date = filter_df.filter(pl.col('Date_Type_CBS').is_in(['Cashflow.Date', 'Date_Filler']))['Date'].max()
        period1 = filter_df.filter(pl.col('rank') == 1)['Period_Days'].max()
        period2 = filter_df.filter(pl.col('rank') == 2)['Period_Days'].max()
        period3 = filter_df.filter(pl.col('rank') == 3)['Period_Days'].max()
        period4 = filter_df.filter(pl.col('rank') == 4)['Period_Days'].max()
        period5 = filter_df.filter(pl.col('rank') == 5)['Period_Days'].max()
        period6 = filter_df.filter(pl.col('rank') == 6)['Period_Days'].max()
        expense = filter_df.select(pl.sum("Expense"))
        eir = filter_df.select(pl.sum("EIR_Expense"))
        air = filter_df.filter(~pl.col('rank').is_in([distinct_rank[0]]) & pl.col('Repayment.Amount') == 0, 'Expense').sum()
        air_eir = filter_df.filter(~pl.col('rank').is_in([distinct_rank[0]]) & pl.col('Repayment.Amount') == 0, 'EIR_Expense').sum()

        dates.append(date)
        period1_l.append(period1)
        period2_l.append(period2)
        period3_l.append(period3)
        period4_l.append(period4)
        period5_l.append(period5)
        period6_l.append(period6)
        expense_l.append(expense)
        expense_eir_l.append(eir)
        air_l.append(air)
        air_eir_l.append(air_eir)
        
    Y = Y.with_columns('Cash_Flow_Date', dates)
    Y = Y.with_columns('Period 1', period1_l)
    Y = Y.with_columns('Period 2', period2_l)
    Y = Y.with_columns('Period 3', period3_l)
    Y = Y.with_columns('Period 4', period4_l)
    Y = Y.with_columns('Period 5', period5_l)
    Y = Y.with_columns('Period 6', period6_l)
    Y = Y.with_columns('Y_Expense', expense_l)
    Y = Y.with_columns('Y_Expense_EIR', expense_eir_l)
    Y = Y.with_columns('Y_AIR', air_l)
    Y = Y.with_columns('Y_AIR_EIR', air_eir_l)
    
    CBS_subset['CashFlow.Date'] = CBS_subset['CashFlow.Date'].str.to_datetime()
    temp_df = CBS_subset.join(Y, on='CashFlow.Date', how='left', force_orthogonal=True)
    P11_subset = P11[P11['Date_Type_CBS'].is_in(['Cashflow.Date','Date_Filter'])]
    
    df = df.with_columns((pl.col("Y_Expense_EIR") - pl.col("Y_AIR_EIR")).alias("Intrest"))
    df['Intrest'].fill_nan(0)
    interest_value = 0
    df = df.with_columns(pl.lit(interest_value).alias('Intrest_Amount'))
    
 ####for loop part remaining 
    for i in range(1,len(final_df)):
                final_df['Intrest_Amount'][i] = final_df['Intrest'][i] + final_df['Y_AIR_EIR'][i]
                if final_df['Date_Type'][i] == "Date_Filler":
                    final_df['Y_AIR'][i] = final_df['Y_Expense'][i] + final_df['Y_AIR'][i-1]
                    final_df['Y_AIR_EIR'][i] = final_df['Y_Expense_EIR'][i] + final_df['Y_AIR_EIR'][i-1]
                else:
                    final_df['Y_AIR'][i] = final_df['Y_AIR'][i] 
                    final_df['Y_AIR_EIR'][i] = final_df['Y_AIR_EIR'][i] 



    df = df.with_columns((pl.col("Intrest") - pl.col("Y_AIR_EIR")).alias("Intrest_Amount"))
    
    df = df.with_columns(
        pl.when (pl.col('Date_Type') == "Date_Filler") \
        .then ((pl.col("Y_Expense") - pl.col("Y_AIR")).alias("Y_AIR") & (pl.col( "Intrest_Amount") = 0))  \
        .otherwise ((pl.col('Y_AIR') = pl.col('Y_AIR')) & (pl.col('Y_AIR_EIR') = pl.col('Y_AIR_EIR'))))                
   
    
    df = df.with_columns((pl.col("Repayment.Amount") - pl.col("Intrest_Amount")).alias("Principal Installment"))
                        
    df = df.with_columns(
        pl.when (pl.col('Repayment.Amount') == 0) \
        .then ((pl.col('Principal Installment') = 0) & (pl.col( "Intrest_Amount")= 0)))                

    
    df = df.with_columns(pl.lit(revised_intrest_rate).alias('Effective_Rate'))
    df['LoanAccount'] = CBS_subset['Loan.Account']         ####### Doubt
    period1_l.append(period1)
    

SyntaxError: invalid syntax (1939437108.py, line 80)

In [ ]:
['date', 'fine']

In [ ]:
df = pl.read_csv('Dummy_EIR_Test_Data.csv')

In [ ]:
df.head()

In [ ]:
df = df.with_columns((pl.col("REPAYMENT_AMOUNT") - pl.col("CURRENCY_ID")).alias("a^2"))
df['a^2'].fill_nan(0)

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
df = df.withColumn("users", df["users"].cast(IntegerType()))
df = df.withColumn("sales", df["sales"].cast(IntegerType()))
df['buy_rate'] = df['sales']/ df['users']

In [ ]:
df['PRINCIPAL_AMOUNT'].max()

In [ ]:
df.filter(pl.col("DAY_COUNT_BASIS").is_in

In [ ]:
final_df['Intrest'] = final_df['Y_Expense_EIR'] - final_df['Y_AIR_EIR']